In [1]:
import torch
import torchvision
from utils.visualise import make_dot

In [2]:
resnet_18 = torchvision.models.resnet18(pretrained=True)
resnet_18.eval();

In [3]:
# by using the context manager "with torch.no_grad():", intermediate caches are not saved
# making the inspection of the graph pretty boring / useless
torch.manual_seed(0)
x = torch.randn(1, 3, 224, 224)
h_x = resnet_18(x)

In [4]:
dot = make_dot(h_x)  # generate network graph
dot.render('net.dot');  # save DOT and PDF in the current directory
# dot  # uncomment for displaying the graph in the notebook

In [5]:
# explore network graph
print('h_x creator ->',h_x.grad_fn)
print('h_x creator prev fun type ->', type(h_x.grad_fn.next_functions))
print('h_x creator prev fun length ->', len(h_x.grad_fn.next_functions))
print('\n--- content of h_x creator prev fun ---')
for a, b in enumerate(h_x.grad_fn.next_functions): print(a, '-->', b)
print('---------------------------------------\n')

h_x creator -> <ThAddmmBackward object at 0x122f55dd8>
h_x creator prev fun type -> <class 'tuple'>
h_x creator prev fun length -> 3

--- content of h_x creator prev fun ---
0 --> (<ExpandBackward object at 0x111434630>, 0)
1 --> (<ViewBackward object at 0x111434e80>, 0)
2 --> (<TBackward object at 0x1233ca358>, 0)
---------------------------------------



The current node is a `torch.nn._functions.linear.Linear` object, fed by

- 0 --> output of `torch.autograd._functions.tensor.View` object
- 1 --> weight matrix of size `(1000, 512)`
- 2 --> bias vector of size `(1000)`

In [6]:
print(resnet_18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [7]:
resnet_18._modules.keys()

odict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])

In [8]:
avgpool_layer = resnet_18._modules.get('avgpool')
h = avgpool_layer.register_forward_hook(
        lambda m, i, o: \
        print(
            'm:', type(m),
            '\ni:', type(i),
                '\n   len:', len(i),
                '\n   type:', type(i[0]),
                '\n   data size:', i[0].data.size(),
                '\n   data type:', i[0].data.type(),
            '\no:', type(o),
                '\n   data size:', o.data.size(),
                '\n   data type:', o.data.type(),
        )
)
h_x = resnet_18(x)
h.remove()

m: <class 'torch.nn.modules.pooling.AvgPool2d'> 
i: <class 'tuple'> 
   len: 1 
   type: <class 'torch.Tensor'> 
   data size: torch.Size([1, 512, 7, 7]) 
   data type: torch.FloatTensor 
o: <class 'torch.Tensor'> 
   data size: torch.Size([1, 512, 1, 1]) 
   data type: torch.FloatTensor


In [9]:
my_embedding = torch.zeros(512)
def fun(m, i, o): my_embedding.copy_(o.data.squeeze())
h = avgpool_layer.register_forward_hook(fun)
h_x = resnet_18(x)
h.remove()

In [10]:
# print first values of the embedding
my_embedding[:9]

tensor([0.2126, 0.0000, 0.0850, 3.1023, 0.1199, 0.0000, 0.0098, 1.4544, 1.0311])